In [1]:
import pandas as pd
import urllib3
import zipfile
import shutil
import os
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
    
http = urllib3.PoolManager()
url = 'http://www.manythings.org/anki/fra-eng.zip'
filename = 'fra-eng.zip'
path = os.getcwd()
zipfilename = os.path.join(path,filename)
with http.request('GET', url, preload_content=False) as r, open(zipfilename, 'wb') as out_file:
    shutil.copyfileobj(r, out_file)
    
with zipfile.ZipFile(zipfilename, 'r') as zip_ref:
    zip_ref.extractall(path)

In [2]:
lines = pd.read_csv('fra.txt', names=['src', 'tar', 'lic'], sep='\t')
del lines['lic']
len(lines)

179904

In [3]:
lines = lines.loc[:, 'src':'tar']
lines = lines[0:60000] # 6만개만 저장
lines.sample(10)

,src,tar
21573,She is attractive.,Elle est attirante.
32385,Put some clothes on.,Habillez-vous !
43153,I guess the dog bites.,Je crois que le chien mord.
43090,I find you attractive.,Je vous trouve attirantes.
33851,We'll never do that.,Nous ne ferons jamais ça.
40170,We're getting closer.,Nous nous approchons.
33562,Tom was a great guy.,Tom était un mec génial.
51952,Stop talking like that.,Arrêtez de parler comme ça.
41795,Does Tom have a beard?,Tom a-t-il une barbe ?
54900,Did I hurt his feelings?,L'ai-je froissé ?


In [4]:
lines.tar = lines.tar.apply(lambda x : '\t '+ x + ' \n')
lines.sample(10)

,src,tar
2018,Ask anybody.,\t Demande à qui que ce soit ! \n
22042,They are very big.,\t Elles sont très grandes. \n
21708,Stop overreacting.,\t Cessez de réagir de façon excessive. \n
24386,Go and wake her up.,\t Va la réveiller. \n
21538,She decided to go.,\t Elle a décidé de partir. \n
46854,What do you call this?,\t Comment appelles-tu ceci ? \n
6350,I'm your boss.,\t Je suis ton chef. \n
36632,I hated you at first.,"\t Je vous détestais, au début. \n"
18524,You're assertive.,\t Tu sais te faire comprendre. \n
206,Kiss me.,\t Embrassez-moi. \n


In [5]:
# 글자 집합 구축
src_vocab=set()
for line in lines.src: # 1줄씩 읽음
    for char in line: # 1개의 글자씩 읽음
        if char not in src_vocab:
            src_vocab.add(char)

tar_vocab=set()
for line in lines.tar:
    for char in line:
        if char not in tar_vocab:
            tar_vocab.add(char)

In [6]:
src_vocab_size = len(src_vocab)
tar_vocab_size = len(tar_vocab)
print(src_vocab_size)
print(tar_vocab_size)

78
105


In [7]:
src_to_index = dict([(word, i) for i, word in enumerate(src_vocab)])
tar_to_index = dict([(word, i) for i, word in enumerate(tar_vocab)])
print(src_to_index)
print(tar_to_index)

{'g': 0, '%': 1, 'F': 2, 'S': 3, 'x': 4, 'L': 5, '!': 6, '5': 7, '1': 8, 'R': 9, ':': 10, 'B': 11, 'b': 12, 'r': 13, 'D': 14, '-': 15, 'h': 16, 'K': 17, 'j': 18, '"': 19, '6': 20, 'z': 21, '.': 22, 'T': 23, 'X': 24, ',': 25, 'c': 26, 'a': 27, 'O': 28, '4': 29, 'k': 30, 'd': 31, 'A': 32, 'i': 33, 'u': 34, 'n': 35, 'y': 36, '8': 37, 'U': 38, '€': 39, 'V': 40, 'e': 41, ' ': 42, 'Q': 43, '’': 44, 'E': 45, '3': 46, 'o': 47, '0': 48, "'": 49, 'W': 50, 'q': 51, '$': 52, 'Z': 53, 'm': 54, '7': 55, 'é': 56, 'P': 57, 'w': 58, 'N': 59, 's': 60, '2': 61, 'I': 62, '/': 63, 'M': 64, 'v': 65, '&': 66, 'G': 67, 'C': 68, '9': 69, 'Y': 70, 't': 71, 'J': 72, '?': 73, 'l': 74, 'f': 75, 'H': 76, 'p': 77}
{'g': 0, '%': 1, 'F': 2, 'S': 3, 'x': 4, '\u200b': 5, '\t': 6, 'L': 7, '!': 8, '\u202f': 9, '5': 10, '(': 11, '1': 12, 'R': 13, 'û': 14, ':': 15, 'B': 16, 'b': 17, 'r': 18, '-': 19, 'D': 20, 'h': 21, 'K': 22, '\n': 23, 'É': 24, 'Ç': 25, 'j': 26, '"': 27, 'ë': 28, '6': 29, 'ï': 30, 'z': 31, '.': 32, 'T': 33

In [8]:
encoder_input = []
for line in lines.src: #입력 데이터에서 1줄씩 문장을 읽음
    temp_X = []
    for w in line: #각 줄에서 1개씩 글자를 읽음
        temp_X.append(src_to_index[w]) # 글자를 해당되는 정수로 변환
    encoder_input.append(temp_X)
print(encoder_input[:5])

[[67, 47, 22], [76, 33, 22], [76, 33, 22], [9, 34, 35, 6], [9, 34, 35, 6]]


In [9]:
decoder_input = []
for line in lines.tar:
    temp_X = []
    for w in line:
        temp_X.append(tar_to_index[w])
    decoder_input.append(temp_X)
print(decoder_input[:5])

[[6, 58, 57, 37, 58, 8, 58, 23], [6, 58, 3, 37, 101, 44, 96, 58, 8, 58, 23], [6, 58, 3, 37, 101, 44, 96, 32, 58, 23], [6, 58, 91, 64, 44, 18, 83, 9, 8, 58, 23], [6, 58, 91, 64, 44, 18, 59, 31, 9, 8, 58, 23]]


In [10]:
decoder_target = []
for line in lines.tar:
    t=0
    temp_X = []
    for w in line:
        if t>0:
            temp_X.append(tar_to_index[w])
        t=t+1
    decoder_target.append(temp_X)
print(decoder_target[:5])

[[58, 57, 37, 58, 8, 58, 23], [58, 3, 37, 101, 44, 96, 58, 8, 58, 23], [58, 3, 37, 101, 44, 96, 32, 58, 23], [58, 91, 64, 44, 18, 83, 9, 8, 58, 23], [58, 91, 64, 44, 18, 59, 31, 9, 8, 58, 23]]


In [11]:
max_src_len = max([len(line) for line in lines.src])
max_tar_len = max([len(line) for line in lines.tar])
print(max_src_len)
print(max_tar_len)

24
76


In [12]:
encoder_input = pad_sequences(encoder_input, maxlen=max_src_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen=max_tar_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen=max_tar_len, padding='post')

In [13]:
encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_target = to_categorical(decoder_target)

In [14]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model
import numpy as np

encoder_inputs = Input(shape=(None, src_vocab_size))
encoder_lstm = LSTM(units=256, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
# encoder_outputs도 같이 리턴받기는 했지만 여기서는 필요없으므로 이 값은 버림.
encoder_states = [state_h, state_c]
# LSTM은 바닐라 RNN과는 달리 상태가 두 개. 바로 은닉 상태와 셀 상태.

decoder_inputs = Input(shape=(None, tar_vocab_size))
decoder_lstm = LSTM(units=256, return_sequences=True, return_state=True)
decoder_outputs, _, _= decoder_lstm(decoder_inputs, initial_state=encoder_states)
# 디코더의 첫 상태를 인코더의 은닉 상태, 셀 상태로 합니다.
decoder_softmax_layer = Dense(tar_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="rmsprop", loss="categorical_crossentropy")

In [15]:
model.fit(x=[encoder_input, decoder_input], y=decoder_target, batch_size=32, epochs=50, validation_split=0.2)

Train on 48000 samples, validate on 12000 samples
Epoch 1/50
48000/48000 [==============================] - 19s 400us/sample - loss: 0.6455 - val_loss: 0.5898
Epoch 2/50
48000/48000 [==============================] - 14s 301us/sample - loss: 0.4042 - val_loss: 0.4824
Epoch 3/50
48000/48000 [==============================] - 15s 304us/sample - loss: 0.3400 - val_loss: 0.4323
Epoch 4/50
48000/48000 [==============================] - 15s 306us/sample - loss: 0.3044 - val_loss: 0.4042
Epoch 5/50
48000/48000 [==============================] - 15s 305us/sample - loss: 0.2808 - val_loss: 0.3872
Epoch 6/50
48000/48000 [==============================] - 15s 304us/sample - loss: 0.2637 - val_loss: 0.3742
Epoch 7/50
48000/48000 [==============================] - 15s 309us/sample - loss: 0.2502 - val_loss: 0.3684
Epoch 8/50
48000/48000 [==============================] - 15s 308us/sample - loss: 0.2390 - val_loss: 0.3626
Epoch 9/50
48000/48000 [==============================] - 15s 310us/sample - l

In [17]:
# Seq2seq 기계 번역기 동작시키기
encoder_model = Model(inputs=encoder_inputs, outputs=encoder_states)

# 이전 시점의 상태들을 저장하는 텐서
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
# 문장의 다음 단어를 예측하기 위해서 초기 상태(initial state)를 이전 시점의 상태로 사용. 이는 뒤의 함수 decode_sequence()에 구현.
decoder_states = [state_h, state_c]
# 훈련 과정에서와 달리 LSTM이 리턴하는 은닉 상태와 셀 상태인 state_h와 state_c를 버리지 않음.
decoder_outputs = decoder_softmax_layer(decoder_outputs)
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs] + decoder_states)

# index로부터 단어를 얻을 수 있는 변수
index_to_src = dict((i, char) for char, i in src_to_index.items())
index_to_tar = dict((i, char) for char, i in tar_to_index.items())


In [18]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)
    
    # <SOS>에 해당하는 원-핫 벡터 생성
    target_seq = np.zeros((1, 1, tar_vocab_size))
    target_seq[0, 0, tar_to_index['\t']] = 1. # \t = 6, \n = 23, \t 위치의 값을 1. 으로 바꿔주어 원-핫 벡터를 만든다는 의미.
    
    stop_condition = False
    decoded_sentence = ""
    
    # stop_condition이 True가 될 때까지 루프 반복
    while not stop_condition:
        # 이전 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        
        # 예측 결과를 문자로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = index_to_tar[sampled_token_index]
        
        # 현재 시점의 예측 문자를 예측 문장에 추가
        decoded_sentence += sampled_char
        
        # <eos>에 도달하거나 최대 길이를 넘으면 중단.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_tar_len):
            stop_condition = True
            
        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1, 1, tar_vocab_size))
        target_seq[0, 0, sampled_token_index] = 1.
        
        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]
        
    return decoded_sentence

In [19]:
for seq_index in [3, 50, 100, 300, 1001]: # 입력 문장의 인덱스
    input_seq = encoder_input[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print(35 * "-")
    print('입력 문장:', lines.src[seq_index])
    print('정답 문장:', lines.tar[seq_index][1:len(lines.tar[seq_index])-1]) # 앞뒤로 \t와 \n 제거하고 출력
    print('번역기가 번역한 문장:', decoded_sentence[:len(decoded_sentence)-1]) # 마지막에 \n 빼고 출력

-----------------------------------
입력 문장: Run!
정답 문장:  Cours ! 
번역기가 번역한 문장:  Consez ! 
-----------------------------------
입력 문장: I left.
정답 문장:  Je suis parti. 
번역기가 번역한 문장:  J'ai compris. 
-----------------------------------
입력 문장: Burn it.
정답 문장:  Brûlez-la. 
번역기가 번역한 문장:  Brûle-le. 
-----------------------------------
입력 문장: Drive on.
정답 문장:  Continue à rouler ! 
번역기가 번역한 문장:  Avancez ! 
-----------------------------------
입력 문장: Step back.
정답 문장:  Recule ! 
번역기가 번역한 문장:  Reculez! Nous finier ! 
